In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# HSMMの性能比較の数値実験

In [2]:
import sys
sys.path.append("../lib")

In [59]:
import math

from IPython.core.display import display, Markdown, Latex
import numpy as np
from scipy.special import gammaln, psi
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, t, cauchy, laplace, gumbel_r, gamma, skewnorm, pareto, multivariate_normal
from typing import Callable
from sklearn.mixture import BayesianGaussianMixture
from sklearn.datasets import load_iris, load_wine

from HyperbolicSecantMixtureModelVB import HyperbolicSecantMixtureVB
from learning import GaussianMixtureModelVB
from util import GaussianMixtureModel, HyperbolicSecantMixtureModel, StudentMixtureModel, LaplaceMixtureModel, GumbelMixtureModel

# 問題設定

## 真の分布の設定
+ データ生成分布は変更しますが、混合比, 中心, scaleは同じものを流用

In [4]:
true_ratio = np.array([0.33, 0.33, 0.34])
true_delta = 0
true_s = np.array([[2, 2], [0.5, 0.5], [1, 1]])
true_b = np.array([[2, 4], [-4, -2], [0, 0]])
true_param = dict()
true_param["ratio"] = true_ratio
true_param["mean"] = true_b
true_param["precision"] = true_s
true_param["scale"] = np.array([np.diag(1/np.sqrt(true_s[k,:])) for k in range(len(true_ratio))])
K0 = len(true_ratio)
M = true_b.shape[1]

## Learning setting:

In [5]:
### 学習データの数
n = 400

### テストデータの数
N = 10000

### データの出方の個数
ndataset = 50

### 事前分布のハイパーパラメータ
pri_params = {
    "pri_alpha": 0.1,
    "pri_beta": 0.001,
    "pri_gamma": M+2,
    "pri_delta": 1
}

### データ生成の回数
data_seed_start = 201907
data_seeds = np.arange(start = data_seed_start, stop = data_seed_start + ndataset, step = 1)

### 学習モデルの初期値の乱数 -> データseedにoffsetを加えたものを使う
learning_num = 10
learning_seed_offset = 100

### 繰り返しアルゴリズムの繰り返し回数
learning_iteration = 1000

### 学習モデルのコンポーネントの数
K = np.array([3, 5])

# 性能評価
+ 1連の流れ
    1. データ生成する
    1. 学習を行う
    1. 精度評価を行う
    1. 1に戻って再度計算

# コンポーネントの分布が正規分布の場合

In [6]:
gerror_gmm_diag = np.zeros(len(data_seeds))
cklerror_gmm_diag = np.zeros(len(data_seeds))
c01error_gmm_diag = np.zeros(len(data_seeds))

gerror_gmm_cov = np.zeros(len(data_seeds))
cklerror_gmm_cov = np.zeros(len(data_seeds))
c01error_gmm_cov = np.zeros(len(data_seeds))

gerror_hsmm = np.zeros(len(data_seeds))
cklerror_hsmm = np.zeros(len(data_seeds))
c01error_hsmm = np.zeros(len(data_seeds))

for i, data_seed in enumerate(data_seeds):
    ### データを生成する
    (train_X, train_label, train_label_arg) = GaussianMixtureModel.rvs(true_ratio, true_b, true_s, size = n, data_seed = data_seed)
    (test_X, test_label, test_label_arg) = GaussianMixtureModel.rvs(true_ratio, true_b, true_s, size = N)
    
    gmm_diag_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
    gmm_diag_obj.fit(train_X)
    
    gmm_cov_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
    gmm_cov_obj.fit(train_X)
    
    hsmm_obj = HyperbolicSecantMixtureVB(K = K[0],                                     
                                         pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                         iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
    hsmm_obj.fit(train_X)
    posterior_true_logprob = GaussianMixtureModel().latent_posterior_logprob(train_X, true_ratio, true_b, true_s)
    cklerror_gmm_diag[i] = gmm_diag_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_gmm_cov[i] = gmm_cov_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_hsmm[i] = hsmm_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    
    c01error_gmm_diag[i] = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_gmm_cov[i] = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_hsmm[i] = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)
    
    true_empirical_entropy = -GaussianMixtureModel.logpdf(test_X, true_ratio, true_b, true_s)
    gerror_gmm_diag[i] = (-true_empirical_entropy - gmm_diag_obj.predict_logproba(test_X))/len(test_X)
    gerror_gmm_cov[i] = (-true_empirical_entropy - gmm_cov_obj.predict_logproba(test_X))/len(test_X)
    gerror_hsmm[i] = (-true_empirical_entropy - hsmm_obj.predict_logproba(test_X))/len(test_X)

In [7]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag.mean()},
gerror_gmm_cov: {gerror_gmm_cov.mean()},
gerror_hsmm: {gerror_hsmm.mean()},
cklerror_gmm_diag: {cklerror_gmm_diag.mean()},
cklerror_gmm_cov: {cklerror_gmm_cov.mean()},
cklerror_hsmm: {cklerror_hsmm.mean()},
c01error_gmm_diag: {c01error_gmm_diag.mean()},
c01error_gmm_cov: {c01error_gmm_cov.mean()},
c01error_hsmm: {c01error_hsmm.mean()}
""")


gerror_gmm_diag: 0.02279932711705087,
gerror_gmm_cov: 0.026461245602702164,
gerror_hsmm: 0.05473828618846722,
cklerror_gmm_diag: 0.006193283216543171,
cklerror_gmm_cov: 0.010147847371069415,
cklerror_hsmm: 0.016026883738220293,
c01error_gmm_diag: 0.9762499999999998,
c01error_gmm_cov: 0.9752000000000001,
c01error_hsmm: 0.9734



# コンポーネントの分布が双曲線正割分布の場合

In [15]:
gerror_gmm_diag = np.zeros(len(data_seeds))
cklerror_gmm_diag = np.zeros(len(data_seeds))
c01error_gmm_diag = np.zeros(len(data_seeds))

gerror_gmm_cov = np.zeros(len(data_seeds))
cklerror_gmm_cov = np.zeros(len(data_seeds))
c01error_gmm_cov = np.zeros(len(data_seeds))

gerror_hsmm = np.zeros(len(data_seeds))
cklerror_hsmm = np.zeros(len(data_seeds))
c01error_hsmm = np.zeros(len(data_seeds))
for i, data_seed in enumerate(data_seeds):
    ### データを生成する
    (train_X, train_label, train_label_arg) = HyperbolicSecantMixtureModel.rvs(true_ratio, true_b, true_s, size = n, data_seed = data_seed)
    (test_X, test_label, test_label_arg) = HyperbolicSecantMixtureModel.rvs(true_ratio, true_b, true_s, size = N)
    
    gmm_diag_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
    gmm_diag_obj.fit(train_X)
    
    gmm_cov_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
    gmm_cov_obj.fit(train_X)
    
    hsmm_obj = HyperbolicSecantMixtureVB(K = K[0],                                     
                                         pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                         iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
    hsmm_obj.fit(train_X)
    
    posterior_true_logprob = HyperbolicSecantMixtureModel().latent_posterior_logprob(train_X, true_ratio, true_b, true_s)
    cklerror_gmm_diag[i] = gmm_diag_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_gmm_cov[i] = gmm_cov_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_hsmm[i] = hsmm_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    
    c01error_gmm_diag[i] = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_gmm_cov[i] = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_hsmm[i] = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)
    
    true_empirical_entropy = -HyperbolicSecantMixtureModel.logpdf(test_X, true_ratio, true_b, true_s)
    gerror_gmm_diag[i] = (-true_empirical_entropy - gmm_diag_obj.predict_logproba(test_X))/len(test_X)
    gerror_gmm_cov[i] = (-true_empirical_entropy - gmm_cov_obj.predict_logproba(test_X))/len(test_X)
    gerror_hsmm[i] = (-true_empirical_entropy - hsmm_obj.predict_logproba(test_X))/len(test_X)

In [16]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag.mean()},
gerror_gmm_cov: {gerror_gmm_cov.mean()},
gerror_hsmm: {gerror_hsmm.mean()},
cklerror_gmm_diag: {cklerror_gmm_diag.mean()},
cklerror_gmm_cov: {cklerror_gmm_cov.mean()},
cklerror_hsmm: {cklerror_hsmm.mean()},
c01error_gmm_diag: {c01error_gmm_diag.mean()},
c01error_gmm_cov: {c01error_gmm_cov.mean()},
c01error_hsmm: {c01error_hsmm.mean()}
""")


gerror_gmm_diag: -0.6083380371753834,
gerror_gmm_cov: -0.612315429116236,
gerror_hsmm: -0.6456245458137825,
cklerror_gmm_diag: 5.331679934170414,
cklerror_gmm_cov: 5.610586460541836,
cklerror_hsmm: 0.9137027064185334,
c01error_gmm_diag: 0.8671999999999999,
c01error_gmm_cov: 0.8568499999999999,
c01error_hsmm: 0.8957999999999999



# コンポーネントの分布がt分布の場合

In [17]:
gerror_gmm_diag = np.zeros(len(data_seeds))
cklerror_gmm_diag = np.zeros(len(data_seeds))
c01error_gmm_diag = np.zeros(len(data_seeds))

gerror_gmm_cov = np.zeros(len(data_seeds))
cklerror_gmm_cov = np.zeros(len(data_seeds))
c01error_gmm_cov = np.zeros(len(data_seeds))

gerror_hsmm = np.zeros(len(data_seeds))
cklerror_hsmm = np.zeros(len(data_seeds))
c01error_hsmm = np.zeros(len(data_seeds))

true_df = 3
for i, data_seed in enumerate(data_seeds):
    ### データを生成する
    (train_X, train_label, train_label_arg) = StudentMixtureModel.rvs(true_ratio, true_b, true_s, size = n, data_seed = data_seed, df = true_df)
    (test_X, test_label, test_label_arg) = StudentMixtureModel.rvs(true_ratio, true_b, true_s, size = N, df = true_df)
    
    gmm_diag_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
    gmm_diag_obj.fit(train_X)
    
    gmm_cov_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
    gmm_cov_obj.fit(train_X)
    
    hsmm_obj = HyperbolicSecantMixtureVB(K = K[0],                                     
                                         pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                         iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
    hsmm_obj.fit(train_X)
    
    posterior_true_logprob = StudentMixtureModel().latent_posterior_logprob(train_X, true_ratio, true_b, true_s, df = true_df)
    cklerror_gmm_diag[i] = gmm_diag_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_gmm_cov[i] = gmm_cov_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_hsmm[i] = hsmm_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    
    c01error_gmm_diag[i] = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_gmm_cov[i] = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_hsmm[i] = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)
    
    true_empirical_entropy = -StudentMixtureModel.logpdf(test_X, true_ratio, true_b, true_s, df = true_df)
    gerror_gmm_diag[i] = (-true_empirical_entropy - gmm_diag_obj.predict_logproba(test_X))/len(test_X)
    gerror_gmm_cov[i] = (-true_empirical_entropy - gmm_cov_obj.predict_logproba(test_X))/len(test_X)
    gerror_hsmm[i] = (-true_empirical_entropy - hsmm_obj.predict_logproba(test_X))/len(test_X)

In [18]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag.mean()},
gerror_gmm_cov: {gerror_gmm_cov.mean()},
gerror_hsmm: {gerror_hsmm.mean()},
cklerror_gmm_diag: {cklerror_gmm_diag.mean()},
cklerror_gmm_cov: {cklerror_gmm_cov.mean()},
cklerror_hsmm: {cklerror_hsmm.mean()},
c01error_gmm_diag: {c01error_gmm_diag.mean()},
c01error_gmm_cov: {c01error_gmm_cov.mean()},
c01error_hsmm: {c01error_hsmm.mean()}
""")


gerror_gmm_diag: 0.19055017386161752,
gerror_gmm_cov: 0.19093998161244208,
gerror_hsmm: 0.047143602214668894,
cklerror_gmm_diag: 1.326964762751781,
cklerror_gmm_cov: 1.9558436129389745,
cklerror_hsmm: 0.040873788363285486,
c01error_gmm_diag: 0.8486000000000001,
c01error_gmm_cov: 0.7792999999999999,
c01error_hsmm: 0.9077500000000001



# コンポーネントの分布がラプラス分布の場合

In [19]:
gerror_gmm_diag = np.zeros(len(data_seeds))
cklerror_gmm_diag = np.zeros(len(data_seeds))
c01error_gmm_diag = np.zeros(len(data_seeds))

gerror_gmm_cov = np.zeros(len(data_seeds))
cklerror_gmm_cov = np.zeros(len(data_seeds))
c01error_gmm_cov = np.zeros(len(data_seeds))

gerror_hsmm = np.zeros(len(data_seeds))
cklerror_hsmm = np.zeros(len(data_seeds))
c01error_hsmm = np.zeros(len(data_seeds))

for i, data_seed in enumerate(data_seeds):
    ### データを生成する
    (train_X, train_label, train_label_arg) = LaplaceMixtureModel.rvs(true_ratio, true_b, true_s, size = n, data_seed = data_seed)
    (test_X, test_label, test_label_arg) = LaplaceMixtureModel.rvs(true_ratio, true_b, true_s, size = N)
    
    gmm_diag_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
    gmm_diag_obj.fit(train_X)
    
    gmm_cov_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
    gmm_cov_obj.fit(train_X)
    
    hsmm_obj = HyperbolicSecantMixtureVB(K = K[0],                                     
                                         pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                         iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
    hsmm_obj.fit(train_X)
    
    posterior_true_logprob = LaplaceMixtureModel().latent_posterior_logprob(train_X, true_ratio, true_b, true_s)
    cklerror_gmm_diag[i] = gmm_diag_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_gmm_cov[i] = gmm_cov_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_hsmm[i] = hsmm_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    
    c01error_gmm_diag[i] = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_gmm_cov[i] = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_hsmm[i] = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)
    
    true_empirical_entropy = -LaplaceMixtureModel.logpdf(test_X, true_ratio, true_b, true_s)
    gerror_gmm_diag[i] = (-true_empirical_entropy - gmm_diag_obj.predict_logproba(test_X))/len(test_X)
    gerror_gmm_cov[i] = (-true_empirical_entropy - gmm_cov_obj.predict_logproba(test_X))/len(test_X)
    gerror_hsmm[i] = (-true_empirical_entropy - hsmm_obj.predict_logproba(test_X))/len(test_X)

In [20]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag.mean()},
gerror_gmm_cov: {gerror_gmm_cov.mean()},
gerror_hsmm: {gerror_hsmm.mean()},
cklerror_gmm_diag: {cklerror_gmm_diag.mean()},
cklerror_gmm_cov: {cklerror_gmm_cov.mean()},
cklerror_hsmm: {cklerror_hsmm.mean()},
c01error_gmm_diag: {c01error_gmm_diag.mean()},
c01error_gmm_cov: {c01error_gmm_cov.mean()},
c01error_hsmm: {c01error_hsmm.mean()}
""")


gerror_gmm_diag: 0.11165531322790817,
gerror_gmm_cov: 0.10232204767039439,
gerror_hsmm: 0.041312681302911464,
cklerror_gmm_diag: 0.5307592658959779,
cklerror_gmm_cov: 0.7679928267586058,
cklerror_hsmm: 0.014391864075189225,
c01error_gmm_diag: 0.87155,
c01error_gmm_cov: 0.8409500000000001,
c01error_hsmm: 0.9197500000000001



# コンポーネントの分布がガンベル分布の場合

In [21]:
gerror_gmm_diag = np.zeros(len(data_seeds))
cklerror_gmm_diag = np.zeros(len(data_seeds))
c01error_gmm_diag = np.zeros(len(data_seeds))

gerror_gmm_cov = np.zeros(len(data_seeds))
cklerror_gmm_cov = np.zeros(len(data_seeds))
c01error_gmm_cov = np.zeros(len(data_seeds))

gerror_hsmm = np.zeros(len(data_seeds))
cklerror_hsmm = np.zeros(len(data_seeds))
c01error_hsmm = np.zeros(len(data_seeds))

for i, data_seed in enumerate(data_seeds):
    ### データを生成する
    (train_X, train_label, train_label_arg) = GumbelMixtureModel.rvs(true_ratio, true_b, true_s, size = n, data_seed = data_seed)
    (test_X, test_label, test_label_arg) = GumbelMixtureModel.rvs(true_ratio, true_b, true_s, size = N)
    
    gmm_diag_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
    gmm_diag_obj.fit(train_X)
    
    gmm_cov_obj = GaussianMixtureModelVB(K = K[0],
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
    gmm_cov_obj.fit(train_X)
    
    hsmm_obj = HyperbolicSecantMixtureVB(K = K[0],                                     
                                         pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                         iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
    hsmm_obj.fit(train_X)
    
    posterior_true_logprob = GumbelMixtureModel().latent_posterior_logprob(train_X, true_ratio, true_b, true_s)
    cklerror_gmm_diag[i] = gmm_diag_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_gmm_cov[i] = gmm_cov_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    cklerror_hsmm[i] = hsmm_obj.score_latent_kl(posterior_true_logprob)[0]/len(train_X)
    
    c01error_gmm_diag[i] = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_gmm_cov[i] = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
    c01error_hsmm[i] = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)
    
    true_empirical_entropy = -GumbelMixtureModel.logpdf(test_X, true_ratio, true_b, true_s)
    gerror_gmm_diag[i] = (-true_empirical_entropy - gmm_diag_obj.predict_logproba(test_X))/len(test_X)
    gerror_gmm_cov[i] = (-true_empirical_entropy - gmm_cov_obj.predict_logproba(test_X))/len(test_X)
    gerror_hsmm[i] = (-true_empirical_entropy - hsmm_obj.predict_logproba(test_X))/len(test_X)

In [22]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag.mean()},
gerror_gmm_cov: {gerror_gmm_cov.mean()},
gerror_hsmm: {gerror_hsmm.mean()},
cklerror_gmm_diag: {cklerror_gmm_diag.mean()},
cklerror_gmm_cov: {cklerror_gmm_cov.mean()},
cklerror_hsmm: {cklerror_hsmm.mean()},
c01error_gmm_diag: {c01error_gmm_diag.mean()},
c01error_gmm_cov: {c01error_gmm_cov.mean()},
c01error_hsmm: {c01error_hsmm.mean()}
""")


gerror_gmm_diag: 0.12751395425148704,
gerror_gmm_cov: 0.13714497681592203,
gerror_hsmm: 0.12349520762503582,
cklerror_gmm_diag: 0.21204404226924958,
cklerror_gmm_cov: 2.4416294394576115,
cklerror_hsmm: 0.041688194133492375,
c01error_gmm_diag: 0.9226499999999999,
c01error_gmm_cov: 0.8978,
c01error_hsmm: 0.9361999999999999



# For real data
+ Fisher's iris data are used.
+ generalization loss and 01 loss are calculated here.

In [79]:
train_X.mean(axis = 0)

array([5.871, 3.053, 3.831, 1.23 ])

In [89]:
# データを生成する
data = load_iris()
total_data = data.data
mean_val = total_data.mean(axis = 0)
std_val = total_data.std(axis = 0)
total_X = (total_data - mean_val)/std_val

n = 100
N = total_X.shape[0] - n
shuffled_ind = np.random.permutation(n + N)
train_ind = shuffled_ind[:n]
test_ind = shuffled_ind[n:]

train_X = total_X[train_ind,2:]
train_label_arg = data.target[train_ind]
test_X = total_X[test_ind,2:]
test_label_arg = data.target[test_ind,]
n, M = train_X.shape

gmm_diag_obj = GaussianMixtureModelVB(K = K[0],
                                 pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                 iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
gmm_diag_obj.fit(train_X)

gmm_cov_obj = GaussianMixtureModelVB(K = K[0],
                                 pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                 iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
gmm_cov_obj.fit(train_X)

hsmm_obj = HyperbolicSecantMixtureVB(K = K[0],                                     
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
hsmm_obj.fit(train_X)

c01error_gmm_diag = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
c01error_gmm_cov = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
c01error_hsmm = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)

gerror_gmm_diag = - gmm_diag_obj.predict_logproba(test_X)/len(test_X)
gerror_gmm_cov=  - gmm_cov_obj.predict_logproba(test_X)/len(test_X)
gerror_hsmm = - hsmm_obj.predict_logproba(test_X)/len(test_X)

In [90]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag},
gerror_gmm_cov: {gerror_gmm_cov},
gerror_hsmm: {gerror_hsmm},
c01error_gmm_diag: {c01error_gmm_diag},
c01error_gmm_cov: {c01error_gmm_cov},
c01error_hsmm: {c01error_hsmm}
""")


gerror_gmm_diag: 1.0622409949941989,
gerror_gmm_cov: 0.9188617165949486,
gerror_hsmm: 2.078860654802859,
c01error_gmm_diag: 0.95,
c01error_gmm_cov: 0.67,
c01error_hsmm: 0.67



# For real data
+ wine data are used.
+ generalization loss and 01 loss are calculated here.

In [97]:
# データを生成する
data = load_wine()
total_data = data.data
mean_val = total_data.mean(axis = 0)
std_val = total_data.std(axis = 0)
total_X = (total_data - mean_val)/std_val

n = 150
N = total_X.shape[0] - n
shuffled_ind = np.random.permutation(n + N)
train_ind = shuffled_ind[:n]
test_ind = shuffled_ind[n:]

train_X = total_X[train_ind,2:]
train_label_arg = data.target[train_ind]
test_X = total_X[test_ind,2:]
test_label_arg = data.target[test_ind,]
n, M = train_X.shape

gmm_diag_obj = GaussianMixtureModelVB(K = K[1],
                                 pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                 iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "diag")
gmm_diag_obj.fit(train_X)

gmm_cov_obj = GaussianMixtureModelVB(K = K[1],
                                 pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = 15, pri_delta = pri_params["pri_delta"], 
                                 iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset, method = "full")
gmm_cov_obj.fit(train_X)

hsmm_obj = HyperbolicSecantMixtureVB(K = K[1],                                     
                                     pri_alpha = pri_params["pri_alpha"], pri_beta = pri_params["pri_beta"], pri_gamma = pri_params["pri_gamma"], pri_delta = pri_params["pri_delta"], 
                                     iteration = 1000, restart_num=learning_num, learning_seed=data_seed + learning_seed_offset)
hsmm_obj.fit(train_X)

c01error_gmm_diag = gmm_diag_obj.score_clustering(train_label_arg)[0]/len(train_X)
c01error_gmm_cov = gmm_cov_obj.score_clustering(train_label_arg)[0]/len(train_X)
c01error_hsmm = hsmm_obj.score_clustering(train_label_arg)[0]/len(train_X)

gerror_gmm_diag = - gmm_diag_obj.predict_logproba(test_X)/len(test_X)
gerror_gmm_cov=  - gmm_cov_obj.predict_logproba(test_X)/len(test_X)
gerror_hsmm = - hsmm_obj.predict_logproba(test_X)/len(test_X)

In [98]:
print(f"""
gerror_gmm_diag: {gerror_gmm_diag},
gerror_gmm_cov: {gerror_gmm_cov},
gerror_hsmm: {gerror_hsmm},
c01error_gmm_diag: {c01error_gmm_diag},
c01error_gmm_cov: {c01error_gmm_cov},
c01error_hsmm: {c01error_hsmm}
""")


gerror_gmm_diag: 13.431477556597226,
gerror_gmm_cov: 15.858975243742805,
gerror_hsmm: 13.534744234046988,
c01error_gmm_diag: 0.86,
c01error_gmm_cov: 0.5666666666666667,
c01error_hsmm: 0.9

